In [1]:
import pandas as pd
import numpy as np

In [28]:
vehicles = pd.read_csv('data/vehicles/vehicles_30-11-22.csv')
for i in range(1,31):
    if i < 10:
        i = '0'+f'{i}'
    gps = pd.read_csv(f'data/gps_sit/sitgps_2022-10-{i}.csv')
    route_num = '1-37(27)'
    print(f"2022-10-{i}",np.sort(gps[gps.route == route_num].gps_imei.astype(int).unique()))

2022-10-01 []
2022-10-02 []
2022-10-03 []
2022-10-04 [864352047950852 864352047951652]
2022-10-05 [864352047950852 864352047951652]
2022-10-06 [864352047950852 864352047951652]
2022-10-07 [864352047950852 864352047951652]
2022-10-08 []
2022-10-09 []
2022-10-10 []
2022-10-11 []
2022-10-12 []
2022-10-13 []
2022-10-14 []
2022-10-15 []
2022-10-16 []
2022-10-17 []
2022-10-18 []
2022-10-19 [864352047950852 864352047951652]
2022-10-20 [864352047950852 864352047951652]
2022-10-21 [864352047950852 864352047951652]
2022-10-22 [864352047950852 864606049299802 864606049300014 864606049300592
 864606049301467 864606049306201 864606049306417 864606049308595
 864606049310815 864606049310849]
2022-10-23 [864352047951652 864606049299802 864606049300014 864606049300592
 864606049301228 864606049301467 864606049306201 864606049306417
 864606049308595 864606049310815 864606049310849]
2022-10-24 [864606049299802 864606049300014 864606049300592 864606049301228
 864606049301467 864606049306201 86460604930641

In [8]:
# all routes
gps.route.unique()

array(['4-3(17)', '2-38(8)', '4-45(81)', '4-27E(173)', '1-3(34)',
       '4-15(82)', '2-42(44)', '1-2E(34)', '4-23E(140)', '4-17(88)',
       '3-25E(552)', '3-53', '1-61'], dtype=object)

In [22]:
import numpy as np
route_num = '1-37(27)'
np.sort(gps[gps.route == route_num].gps_imei.astype(int).unique())

array([], dtype=int64)

In [23]:
np.sort(vehicles[vehicles.route == route_num].gps_imei.unique())

array([864352047950852, 864352047951652, 864606049299802, 864606049300014,
       864606049300592, 864606049301228, 864606049301467, 864606049306201,
       864606049306417, 864606049308595, 864606049310815, 864606049310849])

# Link Time

In [16]:
arrival = pd.read_csv('/home/ea_admin/Documents/spark/airflow-spark/spark/resources/data/arrival/m08/concat_arrival_time/arrival_time.csv')

In [25]:
arrival = arrival[(arrival.route_num == 6) & (arrival.path == 'go') & (arrival.loop_num == 0) 
            & (arrival.gps_imei == 869530043171871)].sort_values('station_num', ascending=True)

In [72]:
arrival = pd.read_csv('/home/ea_admin/Documents/spark/airflow-spark/spark/resources/data/arrival/m08/concat_arrival_time/arrival_time.csv')

for route in arrival.route_num.unique().tolist():
    for path in arrival.path.unique().tolist():
        df = arrival[(arrival.route_num == route) & (arrival.path == path)]
        unique_imei = df.gps_imei.unique().tolist()
        for imei in unique_imei:
            df = df[(arrival.gps_imei == imei)]
            loops = df.loop_num.unique().tolist()
            for loop in loops:
                df = df[(df.loop_num == loop)].sort_values('station_num', ascending=True)
                time = pd.to_datetime(df.time)
                tmp = (time - time.shift()).dt.total_seconds()
                tmp = tmp.dropna().reset_index(drop=True).reset_index().rename(columns={'index':'start'})
                tmp['end'] = tmp.start + 1
                tmp['link_name'] = tmp.start.astype(str) + '_' + tmp.end.astype(str)

                link_time = tmp.time.tolist()
                link_name = tmp.link_name.tolist()
                link_timestamp = time.iloc[1:]
                path_list = [path] * len(link_time)
                gps_imei_list = [imei] * len(link_time)
                route_list = [route] * len(link_time)

                link = pd.DataFrame([link_name, link_time, link_timestamp, path_list, gps_imei_list, route_list]).T
                link.columns = ['link_name', 'link_time', 'link_timestamp', 'path', 'gps_imei', 'route']
                break
            break
        break
    break
                

/home/ea_admin/miniconda3/envs/bas_env/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [71]:
link

,link_name,link_time,link_timestamp,path,gps_imei,route
0,0_1,165.0,2022-08-01 04:09:28,go,866381052370019,133
1,1_2,30.0,2022-08-01 04:09:58,go,866381052370019,133
2,2_3,30.0,2022-08-01 04:10:28,go,866381052370019,133
3,3_4,45.0,2022-08-01 04:11:13,go,866381052370019,133
4,4_5,60.0,2022-08-01 04:12:13,go,866381052370019,133
...,...,...,...,...,...,...
101,101_102,135.0,2022-08-01 05:15:13,go,866381052370019,133
102,102_103,30.0,2022-08-01 05:15:43,go,866381052370019,133
103,103_104,15.0,2022-08-01 05:15:58,go,866381052370019,133
104,104_105,90.0,2022-08-01 05:17:28,go,866381052370019,133


In [74]:
df

,station_lat,station_long,distance,time,station_num,loop_num,path,route_num,gps_imei,uuid
0,13.551719,100.787562,12.367780,2022-08-01 04:06:43,0,0,go,133,866381052370019,f740a8d4512a4c71bda00e79ff8a9406
1,13.555041,100.774513,51.945667,2022-08-01 04:09:28,1,0,go,133,866381052370019,d99d1a81819f459db7a703848e560a3f
2,13.555705,100.771890,7.657031,2022-08-01 04:09:58,2,0,go,133,866381052370019,4173a9bb6f8b49e99eb04b9241829b7d
3,13.556467,100.768748,72.429582,2022-08-01 04:10:28,3,0,go,133,866381052370019,1b0efe8d088844e291d6c61af19875c0
4,13.557094,100.766329,1.908193,2022-08-01 04:11:13,4,0,go,133,866381052370019,0143ad1c1fe44f2382e0edbdb1c3dc6a
...,...,...,...,...,...,...,...,...,...,...
452,13.713056,100.591332,68.762113,2022-08-01 05:15:13,103,0,go,133,866381052370019,7be4080a3a264b14b10f56dab64ba556
461,13.712668,100.589280,16.161776,2022-08-01 05:15:43,104,0,go,133,866381052370019,b5c04a0980e14b1dbdc87a912ae47800
463,13.712728,100.586812,63.256986,2022-08-01 05:15:58,105,0,go,133,866381052370019,482645265c0549169c1a6af389d9c700
490,13.714850,100.584313,1.571629,2022-08-01 05:17:28,106,0,go,133,866381052370019,ab7b3057edcd43d59b7fe0f945f675e5


In [33]:
link = pd.read_csv('/home/ea_admin/Documents/spark/airflow-spark/spark/resources/data/arrival/m08/links/link_6.csv')
link

,link_name,link_time,link_timestamp,path,gps_imei,routes
0,0_1,90,2022-08-01 05:10:09,go,869530043171871,6
1,1_2,45,2022-08-01 05:10:54,go,869530043171871,6
2,2_3,30,2022-08-01 05:11:24,go,869530043171871,6
3,3_4,60,2022-08-01 05:12:24,go,869530043171871,6
4,4_5,45,2022-08-01 05:13:09,go,869530043171871,6
...,...,...,...,...,...,...
242788,50_51,15,2022-08-31 22:32:15,back,864507039676258,6
242789,51_52,15,2022-08-31 22:32:30,back,864507039676258,6
242790,52_53,15,2022-08-31 22:32:45,back,864507039676258,6
242791,53_54,60,2022-08-31 22:33:45,back,864507039676258,6
